# Understanding Gradient Descent with Python

----

### By the end of this lession you will: 

- Understand the fundamentals of Machine Learning optimization with gradient descent
- Understand basic implementations of gradient and stochastic gradient descent 
- Be able to implement your own simple linear and logistic regression using gradient descent in Python!

## Background and motivation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

## What is gradient descent and why do we use it?

_Gradient descent is an iterative optimization technique for finding the mininimum (or maximum) of a function._

---

### The basics of machine learning -- and why we're talking about Gradient Descent

It's often used in Machine Learning because we're trying to find optimal functions that accomplish tasks given a set of information. Gradient descent is used to 'tweak' a machine learning model by minimizing criteria called 'loss'. Here, 'loss' is the function we'd like to minimize because it's a measure for how 'wrong' our model is on average. And the best way to tell if your accomplishing your task well is by clarifying that you're wrong the least often. 

**Key terms**

* Task → The outcome. Estimating: housing prices given home data (real number), cat or dog given photos (binary outcome)
* Data → Input information to use
* Model → The structure of the algorithm / task
* Loss Function → The model error you want to minimize, or the "wrong-ness" of your model
* Gradient Descent → The optimization algorithm to find the "best" model
* Weights → The parameters that control the influence our input data has on our output data. These are the values our optimization algorithms 'tweaks' to find the lowest "loss", which implies the best "model"

---

## A basic example of a task: estimating household rent

Let's say you wanted to create a model that estimated household rent. The task of your model is to take descriptive qualities of a home, and try to estimate the true rent price. 

### So, how do we do that? A mathematical model for housing price

Let's walk through a basic example model that could take in qualities of a house and give me that real number -- rent price

$$ Y_{pred} = \sum_i^j(X_j*W_j) $$ 

For $j$ features, we estimate some real value as a weighted sum of inputs.

As an example, if we're trying to estimate average household rent in an area, then for every additional bedroom in a house leads to ~$1,000 increase in rent.

**What if I gave my model a 2 bedroom home**:

$$ 2000 = 2*1000,   where {W_1 = 1000}$$ 

If we were to add in, say, bathrooms and found that lead to an average increase in rent by ~$500.

**What if I gave my model a 2 bedroom, 1 bathroom home**:

$$ 2500 = 2*1000 + 1*500,   where {W_1 = 1000, W_2 = 500} $$ 

---

### So, how do we find the _optimal_ matrix or vector $W$?

As we discussed in the section above, we're taking our model, $ Y_{pred} = \sum_i^j(X_j*W_j) $, but we need to find each optimal $W_j$. If you recall, we need some criteria for "optimal" or the "best" model. Well, a simple way to define the best model is to say that the best model is the one that's _wrong the least_. 

Thus, we want a model such that $Y_{pred} ~= Y_{true}$! Or, $Y_{pred} - Y_{true} ~= 0$. One way of defining this is the *Mean Squared Error*: 

$$ MSE := \frac{1}{N} \sum_i^N(Y^{true}_i - Y^{pred}_i)^2 $$

Where $N$ is the number of samples.

If we substitute our simple model there: 

$$ MSE := \frac{1}{N} \sum_i^N(Y^{true}_i - (X*W)_i)^2 $$

Given our input data, $X$ and $W$, we want to get $W$ (out of the many $W$ values out there) to be the minimum of or MSE.

---

## Let's take a look in code!

Let's take our simple model above and create functions for: 

- Model prediction as we've specified above
- Mean squared error

In [ ]:
def model(X, W):
    return np.dot(X, W)

In [ ]:
def mean_squared_error(pred, y):
    return np.mean(np.square(y - pred))

If we generate some data, we can actually plot our error as a function of the outputs of our model!

In [ ]:
# Generating data, could be anything as long as the shapes match up
X = np.ones((50, 1))
W = np.arange(1, 51).reshape(50, 1)
y = np.full((50, 1), 25)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.plot(W, [mean_squared_error(model(xi, wi), yi) for xi, wi, yi in zip(X, W, y)], color='darkorange')
ax.set(xlim=(0, 50), ylim=(-50, 500),
       xlabel="W (Weights)", ylabel="MSE (Loss or Error)",
       title="Mean Squared Error over different Weights");

### What do we need to do find that optimal weight? 

The minimum, given the data we've generated, appears to $W = 25$, but we don't to look at an plot and just guess a value. That's where our algorith comes into play: **gradient descent**

**Gradient descent** relies on derivatives, or **gradients** of functions to find the the minimum of a function. A **derivative** is an operation we take on a function to estimate the "rate of change" or "slope" of a function. This isn't a calculus class, but what we're doing here is using the property that the mininum of a function is the **lowest** value of a function such that the derivative of a function is equal to 0. 

### So, what's the derivative and how does that relate to our error function?

Our error function is what we'd like to minimize, thus, we take the derivative with respect to our weights, $W$, of MSE and we'd get: 

$$ \text{Gradient of MSE} := \frac{2}{N} \sum_i^N X_i*(Y^{true}_i - (X*W)_i) $$

Where we would like $W$ such that:

$$ \frac{2}{N} \sum_i^N X_i*(Y^{true}_i - (X*W)_i)  = 0 $$

In [ ]:
def mse_gradient(pred, X, y):
    return - (2* np.dot(X.T, (y - pred))) / y.size

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.plot(W, [mse_gradient(model(xi, wi), xi, yi) for xi, wi, yi in zip(X, W, y)], color='darkorange')
ax.set(xlim=(20, 30), ylim=(-10, 10),
       xlabel="W (Weights)", ylabel="MSE (Loss or Error)",
       title="Derivative of Mean Squared Error over different Weights");

### What do we need to do find that optimal weight? 

We don't always want to take the exact value of $W$ where the loss is $0$ because we may end up in a **_local minima_**. Thus, what we do is we iteratively move closer and closer to the value of $W$ by moving the **opposite direction of the gradient**, ie. closer to $0$ by just a small step. Thus, we iteratively will confirm that, over time, we will not reach **a** value that is $0$, but true lowest error. 

Let's write a function that implements a version of our gradient descent. Write out a `simple_gradient_descent` function that does exactly that. We will loop over our data, finding **new** values of $W$ that are closest to the gradient being 0 (and loss being 0) by taking a small step in the direction opposite the slope. 

Here, we're going to call that _step_ the **learning rate**. 

In [1]:
def simple_gradient_descent(X, y, learning_rate, iterations):
    
    # Start up list
    W_step = list()
    
    # Generate a random starting weight
    # Note: using a uniform random start, but you don't have to. Just convinient for plotting here.
    W = np.random.uniform(low=0.0, high=50)
    
    # Append to the list of each step
    W_step.append(W)
    
    # How do we iterate and find a better W?
    ## CODE HERE
    
    
    return W_step

In [ ]:
W_step = simple_gradient_descent(X, y, .1, 30)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.scatter(W_step, [mean_squared_error(model(xi, wi), yi) for xi, wi, yi in zip(X, W_step, y)], color='black', s=150)
plt.plot(W, [mean_squared_error(model(xi, wi), yi) for xi, wi, yi in zip(X, W, y)], color='darkorange')
ax.set(xlim=(0, 50), ylim=(-50, 500),
       xlabel="W (Weights)", ylabel="MSE (Loss or Error)",
       title="Gradient Descent in Action!");

for i, txt in enumerate(["step-" + str(i) for i in np.arange(0, 30)]):
    ax.annotate(txt, (W_step[i]+0.5, [mean_squared_error(model(xi, wi), yi) for xi, wi, yi in zip(X, W_step, y)][i]-10))

## Now some randomized data that's closer to "real"

The example implementation of gradient descent above is nice, but it doesn't totally depict reality. When we take a look at real data, it's _never_ so smooth and the error doesn't behave nearly as nicely. This next section will explain why we have to deal with iterations and that learning rate in the first place.

To exhibit something a bit closer to reality, let's generate some random 'semi-linear' data.

In [ ]:
X_rand = np.random.normal(loc=5, scale=15, size=(250,1))
W_true = 8
y_rand = (X_rand * W_true)
y_rand += np.random.normal(loc=0.0, scale=20.0, size=X_rand.shape)

The code above, and subsequent plot below, will show a randomly generated set of data that _appears_ to be about linear. We have some data that's being generated, and the **true** relationship between $X$ and $Y$ is set to be $Y = X*8$. Whatever the data is, the relationship between indicates that $1$ of $X$ leads to $8$ more $Y$.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.scatter(X_rand, y_rand, c='darkorange')
ax.set(xlim=(0, 50), ylim=(0, 300),
       xlabel="Random X", ylabel="Random Y",
       title="Data X seemingly linearly related to Y");

We can create a line based on this to exhibit what an _ideal_ estimator could or _would_ look like if we found that optimal $W$.

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.scatter(X_rand, y_rand, c='darkorange')
plt.plot([min(X_rand), max(X_rand)], [min(X_rand) * W_true, max(X_rand) * W_true], color='black')
ax.set(xlim=(0, 50), ylim=(0, 300),
       xlabel="Random X", ylabel="Random Y",
       title="Data X seemingly linearly related to Y");

### Examining loss with more _noisey_ data

If we take a look at the loss, given we have many $W$ that we could select, the loss will behave much less regularly with data that doesn't follow such a strict, simple relationship.

In [ ]:
W_range = np.linspace(1, 15, num=50)

fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
plt.plot(W_range, [mean_squared_error(model(xi, wi), yi) for xi, wi, yi in zip(X_rand, W_range, y_rand)], c='darkorange')
plt.plot([8, 8], [-10, 30000], c='black')
ax.set(xlim=(0, 50), ylim=(-50, 30000),
       xlabel="W (Weights)", ylabel="MSE (Loss or Error)",
       title="Mean Squared Error over different Weights, black line is the true W");

----

## Now, test this out on some "real" data

Now that we have the fundamentals of this alogirthm down, let's see how it appleis to some real data. From the example above, we can see that it won't quite be so perfectly smooth, but the same principals will apply. 

If we take in a dataset of housing data, let's see if we can learn the optimal weights, given the data, to estiamte the median price of a house in Boston.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [ ]:
def get_housing_dataset_train_test():
    
    dataset_object = load_boston()
    X_features_df = pd.DataFrame(data=dataset_object['data'], 
                                        columns=dataset_object['feature_names'])
    y_labels_df = pd.DataFrame(data=dataset_object['target'], 
                               columns=['target_labels'])
    X_train, X_test, y_train, y_test = train_test_split(X_features_df, y_labels_df, shuffle=True, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_housing_dataset_train_test()

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print(load_boston().DESCR)

### Let's Create our real model function!

Now that we've done a simple Gradient Descent above, let's create a function `model_real_data` that takes in our `X_train, y_train, lr, iterations, and an option for an intercept_bias` so that we can return a `optimal W, list of previous Ws, and list of losses` along the way!

---
**_A quick hint_**:
As a hint on the `intercept` we haven't quite talked about the concept of "bias", but it's an absolute, constant shift in the inputs to add or subtract from the estimated output. Think of it as, in our housing case, adding by default that all house rents are $~600$ regardless of the number of bedrooms and bathrooms on average. Thus, our equation is: 

$$ \text{Rent} = \text{Bedrooms}*1000 + \text{bathrooms}*500 + 600 $$

Thus, what's the value of $X$ here, if our model is $X*W$ for that last $W_0$

$$ \text{Rent} = X_{Bedrooms}*W_2 + X_{bathrooms}*W_1 + X_0*{W_0} $$

And what's a tricky way we can always add on that $X_0$

---

In [ ]:
def model_real_data(X_train, y_train, lr, iterations, intercept_bias=True):
    
    # Setup your losses and weights
    W_step = list()
    loss_step = list()
    
    ## CODE HERE TO FIND THE THE BEST WEIGHTS
    ## RETURN THE BEST W, the list of W_steps, and the list of loss_steps 
    
    return W, W_step, loss_step

### Let's choose a column (or two!) and try it!

Let's choose a column or two from below and set them in `feature_cols`

In [ ]:
X_train.columns

In [ ]:
# Select column strings and enter them in as features!
feature_cols = ['RM']

In [ ]:
W, W_steps, loss_list = model_real_data(X_train[feature_cols].to_numpy(), y_train.to_numpy(), lr=0.01, iterations=100000, intercept_bias=True)

In [ ]:
W

## Compare our results to the _pros_

Let's compare how our algorithm performs against a popular machine learning library like *sci-kit learn*!

**You should be able to just run the code below**

In [ ]:
from sklearn.linear_model import SGDRegressor, Ridge

In [ ]:
sklearn_model = Ridge(alpha=0.0, solver='sag')

In [ ]:
sklearn_model.fit(X_train[feature_cols].to_numpy(), y_train.to_numpy())

In [ ]:
print(sklearn_model.intercept_, sklearn_model.coef_)

Below is another, simpler version of what the above, `Ridge` model is doing as well! 

In [ ]:
sklearn_sgd_model = SGDRegressor(penalty='none', learning_rate='constant', eta0=0.01, fit_intercept=True, max_iter=100000, alpha=0.0)

In [ ]:
sklearn_sgd_model.fit(X_train[feature_cols].to_numpy(), y_train.to_numpy().ravel())

In [ ]:
print(sklearn_sgd_model.intercept_, sklearn_sgd_model.coef_)

## Compare losses

Compare the loss of our models to the others against our **test set**! 

**You should be able to just run the code below**

In [ ]:
W_bias = np.ones((X_test.shape[0], 1))
X = np.concatenate((W_bias, X_test[feature_cols].to_numpy()), axis=1)
our_pred = model(X, W)

In [ ]:
mean_squared_error(our_pred, y_test.to_numpy().ravel())

In [ ]:
mean_squared_error(sklearn_model.predict(X_test[feature_cols].to_numpy()), y_test.to_numpy().ravel())

In [ ]:
mean_squared_error(sklearn_sgd.predict(X_test[feature_cols].to_numpy()), y_test.to_numpy().ravel())

## Closing notes

**What did you discover about:**

- learning rates
- iterations
- coefficients
- losses and loss functions
- trying out different features?